In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mingptf.model import GPT
import numpy as np
import tensorflow as tf
from mingptf.bpe import BPETokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel

2023-01-09 21:27:43.990802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/abhaykumar/opt/anaconda3/envs/mingpt/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2'

In [4]:
if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning


2023-01-09 21:27:52.104309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


number of parameters: 124.44M


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [5]:
tokenizer = BPETokenizer()
hf_tokenizer = GPT2Tokenizer.from_pretrained(model_type)

def generate(prompt='', num_samples=10, steps=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = [[tokenizer.encoder.encoder['<|endoftext|>']]]
        else:
            x = tokenizer(prompt)
    else:
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = hf_tokenizer(prompt, return_tensors='tf').to(device)
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, top_k=40)
    
    out = tokenizer.decode(y.numpy())
    print(out)
        

In [6]:
generate(prompt='Andrej Karpathy, the', steps=20)

Andrej Karpathy, the Russian state representative in the European Parliament, said last Friday that the president had made up his mind.
